## 組合せ最適化問題
量子コンピュータで最適化問題を解くには、イジングモデルといわれる物理モデルを利用する。

## QUBO定式化
QUBOは問題の答えが小さいほうが正解になるように設定された式です。式の形は、

$$
QUBO = -\sum_i h_i q_i -\sum_{i,j}J_{ij}q_iq_j
$$

となっている。iとjは点を表し、hはバイアス（局所磁場）、Jは相互作用と呼ばれます。この式ではqは量子ビットを表し0か1を取ります（イジングの場合は+1か-1）。
私たちはhとJを問題として設定し、qの値を求めます。

## 問題設定の仕方
問題の設定の仕方は、グラフ問題というものに問題を落とすことで計算できますが、いくつか問題を解くことでコツをつかめます。

主に問題のコスト関数は二種類の式を考える必要があります。

１．小さくしたいコスト関数  
２．満たすべき条件（制約条件）

この二つを別々に設計し、つなげることで実装できます。片方しかない式もあります。

In [1]:
!pip install --quiet matplotlib

## set cover(集合被覆問題)

全体集合$U=\{e_1,e_2,e_3,e_4\}$の部分集合$S= \{S_1,S_2,S_3,S_4 \}$
のうち、全体集合Uの要素をすべてカバーする部分集合のうち最小のものを求める。


## QUBO式

問題をバイナリ変数で表す。  
バイナリ変数$y_i$は, 部分集合$S_i$を選択する場合は1、それ以外は0になる変数  
バイナリ変数$x_{a,m}$は、選択した部分集合のうち、要素$a$が$m$個のときは1、そうでない場合は0とする。

QUBO最適化関数は、

$$H = H _ A + H _ B$$
$$
H_{A}=A\sum_{a=1}^{n} (1-\sum_{m}^N x_{a,m})^2 + A\sum_{a=1}^{n} (\sum_{m}^N mx_{a,m}-\sum_{i:a \in V_i}y_{i})^2
$$
$$H_B = B \sum_{i=1}^{N}x_i$$

$n$は部分集合の個数  
$N$は要素の数

### QUBO式の解説

$$
A\sum_{a=1}^{n} (1-\sum_{m}^N x_{a,m})^2
$$

要素数$m$が各要素に対して、ただ1つのみ$x_{a,m}=1$になる条件

$$
A\sum_{a=1}^{n} (\sum_{m}^N mx_{a,m}-\sum_{i:a \in V_i}y_{i})^2
$$
選択した集合がの各要素数が$m$個になる条件

$$
B \sum_{i=1}^{N}x_i
$$

選択する集合を最小化する条件

## 例題

全体集合$U= \{1,2,3,4 \}$  
部分集合  
$S_0=\{1,2\}$  
$S_1=\{2,3\}$  
$S_2=\{3,4\}$  

集合$U$の要素をすべてカバーする、最小の部分集合$S$を選ぶ

In [1]:

import numpy as np

#量子ビット
N=3 #部分集合の数
M=4 #Uの要素数

#q = Array.create('q', shape=M*M+N, vartype='BINARY')

#集合U
U=np.array([1, 2, 3, 4])

#部分集合
S=np.array([[1, 2], #S0
           [2, 3],  #S1
           [3, 4]]) #S2


#QUBO式
HA = 0
for a in range(M): #要素数
    tmp=0
    for m in range(M): 
        tmp += q[M*a+m]
    HA+=(1-tmp)**2
    
HB = 0
for a in range(M):
    tmp = 0
    for m in range(M):
        tmp += (m+1)*q[M*a+m]
    
    for i in range(N):
        for j in range(2):
            if S[i,j]==U[a]:
                tmp += -q[M*M+i]
    HB+=tmp**2
        
HC =0
for i in range(N):
    HC += q[M*M+i]


#式をつなげる
H = HA + HB + HC

Sample = {'q[9]': 0, 'q[7]': 0, 'q[8]': 1, 'q[4]': 1, 'q[17]': 0, 'q[6]': 0, 'q[12]': 1, 'q[16]': 1, 'q[11]': 0, 'q[0]': 1, 'q[3]': 0, 'q[10]': 0, 'q[14]': 0, 'q[5]': 0, 'q[2]': 0, 'q[18]': 1, 'q[13]': 0, 'q[15]': 0, 'q[1]': 0}
Cost = 2.0
Break = {}


結果を確認すると、$S_0$と$S_1$を選択すると、全体集合の要素数をカバーできることが分かる。

## set packing(集合パッキング問題)

全体集合$U=\{e_1,e_2,e_3,e_4\}$の部分集合$S= \{S_1,S_2,S_3,S_4 \}$
のうち、共通の要素を持たない部分集合のうち最大のものを求める。  


### QUBO式

$$
\large
H_A = A\sum_{i,j:V_i \cap V_j \neq \emptyset} x_ix_j 
$$

$$
H_B= -B \sum_i x_i
$$

## 問題設定



In [3]:
import numpy as np

#量子ビット
N=3 #部分集合の数

#q = Array.create('q', shape=N, vartype='BINARY')

#集合U
U=np.array([1, 2, 3, 4])

#部分集合
S=np.array([[1, 2],  #S0
            [2, 3],  #S1
            [3, 4]]) #S2

#QUBO式
HA = 0
for i in range(N):
    for j in range(N):
        c =np.intersect1d(S[i], S[j]) #共通要素
        if i!=j and c.size != 0:
            HA += q[i]*q[j] 
    
HB = 0
for i in range(N):
    HB += -q[i]


#式をつなげる
H = HA + HB

Sample = {'q[2]': 1, 'q[0]': 1, 'q[1]': 0}
Cost = -2.0
Break = {}


$S_0, S_2$を選択する、先ほどと同じ解が得られました。

### 問題2

全体集合$U= \{1,2,3,4,5,6,7,8,9 \}$  
部分集合  
$S_0=\{1,2,3,6,9 \}$  
$S_1=\{1,2,5,8 \}$  
$S_2=\{4,7 \}$  
$S_3=\{4,5 \}$  
$S_4=\{6,9 \}$  

In [111]:
import numpy as np

#量子ビット
N=5 #部分集合の数

#q = Array.create('q', shape=N, vartype='BINARY')

#集合U
U=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

#部分集合
S=np.array([[1,2,3,6,9],[1,2,5,8],[4,7],[4,5],[6,9]])

#QUBO式
HA = 0
for i in range(N):
    for j in range(N):
        c =np.intersect1d(S[i], S[j]) #共通要素
        if i!=j and c.size != 0:
            HA += q[i]*q[j] 
    
HB = 0
for i in range(N):
    HB += -q[i]


#式をつなげる
H = HA + HB

Sample = {'q[0]': 0, 'q[1]': 1, 'q[2]': 1, 'q[3]': 0, 'q[4]': 1}
Cost = -3.0
Break = {}


C:\Users\005087\AppData\Local\Temp\ipykernel_1748\4142535172.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  S=np.array([[1,2,3,6,9],[1,2,5,8],[4,7],[4,5],[6,9]])


In [2]:
import numpy as np

#量子ビット
N=7 #部分集合の数

#q = Array.create('q', shape=N, vartype='BINARY')

#集合U
U=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9 ,10,11,12])

#部分集合
S=np.array([[1,4],[2,5],[3,6],[1,2,3,4,6,7,9],[4,5,7,8],[7,8,9,10,11,12],[6,11]])

#QUBO式
HA = 0
for i in range(N):
    for j in range(N):
        c =np.intersect1d(S[i], S[j]) #共通要素
        if i!=j and c.size != 0:
            HA += q[i]*q[j] 
    
HB = 0
for i in range(N):
    HB += -q[i]


#式をつなげる
H = HA + HB


Sample = {'q[5]': 1, 'q[6]': 0, 'q[1]': 1, 'q[4]': 0, 'q[0]': 1, 'q[3]': 0, 'q[2]': 1}
Cost = -4.0
Break = {}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
